<a href="https://colab.research.google.com/github/Saransh1329/BlackBox-Agentic-AI-for-Predictive-Maintenance/blob/main/ai3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import openpyxl
from google.colab import files

# Upload Excel file
print("Please upload your Excel file:")
uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]

# Load the Excel file
excel_file = pd.ExcelFile(filename)

# Read the sheets
owner_df = pd.read_excel(excel_file, sheet_name='Owner Details')
appointment_df = pd.read_excel(excel_file, sheet_name='Appointment')

print(f"\nLoaded {len(owner_df)} records from Owner Details")
print(f"Loaded {len(appointment_df)} records from Appointment")

# Convert dates to standard format (YYYY-MM-DD) for matching
owner_df['assigned_slot_date'] = pd.to_datetime(owner_df['assigned_slot_date'], errors='coerce').dt.date
appointment_df['date'] = pd.to_datetime(appointment_df['date'], errors='coerce').dt.date

# Normalize time formats
def normalize_time(t):
    """Convert time to HH:MM:SS string format"""
    if pd.isna(t):
        return None
    if isinstance(t, str):
        # Already a string, ensure consistent format
        t = t.strip()
        if len(t) == 5:  # HH:MM
            t = t + ':00'
        return t
    # If it's a time object
    return t.strftime('%H:%M:%S') if hasattr(t, 'strftime') else str(t)

owner_df['assigned_slot_time'] = owner_df['assigned_slot_time'].apply(normalize_time)
appointment_df['slot_time'] = appointment_df['slot_time'].apply(normalize_time)

# Filter cars with assigned slots
cars_to_schedule = owner_df[
    owner_df['assigned_slot_date'].notna() &
    owner_df['assigned_slot_time'].notna() &
    owner_df['assigned_technician'].notna()
].copy()

print(f"\nFound {len(cars_to_schedule)} cars with assigned slots")
print("="*70)

# Add booking_status column to owner_df
owner_df['booking_status'] = ''

# Create appointment lookup dictionary
appointment_dict = {}
for idx, row in appointment_df.iterrows():
    key = f"{row['service_center']}_{row['date']}_{row['slot_time']}"
    appointment_dict[key] = idx

print(f"\nCreated {len(appointment_dict)} appointment slots")
print("\nSample appointment keys (first 3):")
for i, key in enumerate(list(appointment_dict.keys())[:3]):
    print(f"  {key}")
print()

# Process each car one by one
booked_count = 0
reschedule_count = 0

for idx, car in cars_to_schedule.iterrows():
    service_center = car['assigned_service_center']
    date = car['assigned_slot_date']
    slot_time = car['assigned_slot_time']
    technician = car['assigned_technician']

    print(f"\nProcessing: {car['owner_name']} - {car['brand']} {car['model']}")
    print(f"  Vehicle ID: {car['vehicle_id']}")
    print(f"  Assigned: {service_center} | {date} | {slot_time} | {technician}")

    # Find matching appointment slot
    key = f"{service_center}_{date}_{slot_time}"
    print(f"  Looking for key: {key}")

    if key not in appointment_dict:
        print(f"  ⚠ No matching appointment slot found")
        print(f"  Status: RESCHEDULE")
        owner_df.at[idx, 'booking_status'] = 'Reschedule'
        reschedule_count += 1
        continue

    appt_idx = appointment_dict[key]

    # Extract technician number - handle various formats
    tech_str = str(technician).strip()
    if 'Technician_' in tech_str:
        tech_num = tech_str.split('_')[-1].strip()
    elif 'Technician' in tech_str:
        tech_num = tech_str.replace('Technician', '').strip()
    else:
        tech_num = tech_str

    tech_column = f"Technician_{tech_num}"

    # Check if technician column exists
    if tech_column not in appointment_df.columns:
        print(f"  ⚠ Technician column '{tech_column}' not found")
        print(f"  Available columns: {[c for c in appointment_df.columns if 'Technician' in c]}")
        print(f"  Status: RESCHEDULE")
        owner_df.at[idx, 'booking_status'] = 'Reschedule'
        reschedule_count += 1
        continue

    # Check if technician is already booked
    current_status = appointment_df.at[appt_idx, tech_column]

    if pd.notna(current_status) and str(current_status).lower() == 'booked':
        print(f"  ⚠ {tech_column} already booked")
        print(f"  Status: RESCHEDULE")
        owner_df.at[idx, 'booking_status'] = 'Reschedule'
        reschedule_count += 1
    else:
        # Book the technician
        appointment_df.at[appt_idx, tech_column] = 'Booked'
        owner_df.at[idx, 'booking_status'] = 'Booked'
        print(f"  ✓ Successfully booked with {tech_column}")
        print(f"  Status: BOOKED")
        booked_count += 1

    print("-"*70)

# Print summary
print("\n" + "="*70)
print("PROCESSING COMPLETE!")
print("="*70)
print(f"✓ Successfully Booked: {booked_count}")
print(f"⚠ Need Reschedule: {reschedule_count}")
print(f"📊 Total Processed: {len(cars_to_schedule)}")
print("="*70)

# Save to new Excel file
output_filename = 'scheduled_appointments.xlsx'

with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
    owner_df.to_excel(writer, sheet_name='Owner Details', index=False)
    appointment_df.to_excel(writer, sheet_name='Appointment', index=False)

    # Copy other sheets if they exist
    for sheet_name in excel_file.sheet_names:
        if sheet_name not in ['Owner Details', 'Appointment']:
            df = pd.read_excel(excel_file, sheet_name=sheet_name)
            df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n✓ Output saved as '{output_filename}'")

# Download the file
files.download(output_filename)
print("\n✓ File download initiated!")

Please upload your Excel file:


Saving updated_owner_assignments.xlsx to updated_owner_assignments (3).xlsx

Loaded 1970 records from Owner Details
Loaded 1120 records from Appointment

Found 389 cars with assigned slots

Created 1120 appointment slots

Sample appointment keys (first 3):
  Mahindra Service Zone Mohali_2025-12-05_09:00:00
  Mahindra Service Zone Mohali_2025-12-05_10:00:00
  Mahindra Service Zone Mohali_2025-12-05_11:00:00


Processing: Karan Singh - Mahindra Mahindra Scorpio
  Vehicle ID: DL95HF2679
  Assigned: Mahindra Service Zone Derabassi | 2025-12-05 | 16:00:00 | Technician 2
  Looking for key: Mahindra Service Zone Derabassi_2025-12-05_16:00:00
  ✓ Successfully booked with Technician_2
  Status: BOOKED
----------------------------------------------------------------------

Processing: Simran Gupta - Mahindra Mahindra KUV100
  Vehicle ID: MH30NA4257
  Assigned: Mahindra First Choice Panchkula | 2025-12-08 | 17:00:00 | Technician 6
  Looking for key: Mahindra First Choice Panchkula_2025-12-08_17:0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ File download initiated!
